In [8]:
// Import max-plus algebra functions from our library
import { 
  oplus, otimes, odot, transpose, trace, star, plus, powOtimes,
  EPS, INF, Matrix, Vector,
  createEpsilonMatrix, createIdentityMatrix, reset as resetMatrix
} from './max-plus';

console.log("Max-plus algebra library imported successfully");


Max-plus algebra library imported successfully


### SenseToAct

<img src="../../img/SenseToAct.svg" alt="drawing" width="500"/>

In [9]:
// System Max-Plus Encoding
// System setup for Sensor-Processor-Actuator pipeline

// Create matrices for system
let x: Matrix = [[EPS, EPS, EPS]];     // Physical firing times: [sensor, processor, actuator]

// Single logical time variable, then "blast" to 1x3 vector (only sensor scheduled)
let t: number = EPS;
let tBar: Matrix = [[t, t, t]];

// Indices for system
const sensor = 0;    // Sensor reactor
const processor = 1; // Processor reactor  
const actuator = 2;  // Actuator reactor

// Execution times (diagonal of B)
const eDiag: Matrix = [[0.1, 0.1, 0.1]];  // [e_sensor, e_processor, e_actuator]

// Sensor period (1 unit)
const sensorPeriod = 1;

console.log("System setup complete");


System setup complete


In [10]:
// Evolution equations for system in factored form: x' = (GammaStar ⊗ B) ⊗ x ⊕ (GammaStar ⊗ tBar)
// GammaStar: transitive closure of reaction dependencies (no execution times)
// B: diagonal matrix of execution times

// GammaStar for linear chain sensor -> processor -> actuator
// const GammaStar: Matrix = [
//   [0, EPS, EPS],  // sensor reaches itself
//   [eDiag[0][sensor], 0, EPS],  // processor reachable from sensor and itself
//   [eDiag[0][sensor] + eDiag[0][processor], eDiag[0][processor], 0],  // actuator reachable from sensor, processor, and itself
// ];
const Gamma: Matrix = [
  [0, EPS, EPS],  // sensor reaches itself
  [eDiag[0][sensor], 0, EPS],  // processor reachable from sensor and itself
  [EPS, eDiag[0][processor], 0],  // actuator reachable from sensor, processor, and itself
];
const GammaStar: Matrix = otimes(Gamma, Gamma);

// B is diagonal with execution times on the diagonal
const B: Matrix = [
  [eDiag[0][sensor], EPS,                      EPS],
  [EPS,              eDiag[0][processor],      EPS],
  [EPS,              EPS,                      eDiag[0][actuator]],
];

console.log("Gamma:", Gamma);
console.log("GammaStar:", GammaStar);
console.log("B (diagonal):", B);


Gamma: [
  [ 0, -Infinity, -Infinity ],
  [ 0.1, 0, -Infinity ],
  [ -Infinity, 0.1, 0 ]
]
GammaStar: [ [ 0, -Infinity, -Infinity ], [ 0.1, 0, -Infinity ], [ 0.2, 0.1, 0 ] ]
B (diagonal): [
  [ 0.1, -Infinity, -Infinity ],
  [ -Infinity, 0.1, -Infinity ],
  [ -Infinity, -Infinity, 0.1 ]
]


In [11]:
// Step function for system evolution using GammaStar and B
function step(x: Matrix, tBar: Matrix): Matrix {
  const xT: Matrix = transpose(x);
  const tT: Matrix = transpose(tBar);

  // (GammaStar ⊗ B) ⊗ x
  const GB = otimes(GammaStar, B);
  const GBx = otimes(GB, xT);

  // GammaStar ⊗ tBar
  const Gt = otimes(GammaStar, tT);

  // Combine and transpose back to row vector
  const result = oplus(GBx, Gt);
  return transpose(result);
}

// Reporting function (updated to work with matrices)
function reportStats(k: number, x: Matrix, t: number): void {
  const lagK = x[0].map(xi => xi - t); // lag

  console.log(`Tag index k=${k}`);
  console.log(`t(k)= ${t}`);
  console.log(`x(k) = earliest possible firing times = [${x[0].join(', ')}]`);
  console.log(`lag(k) = [${lagK.join(', ')}]`);
}  

console.log("System functions defined");


System functions defined


In [12]:
// System simulation setup
// Initial logical time (scalar then blasted)
t = 0;
tBar = [[t, t, t]];

// Reset x to eps using the library function
x = resetMatrix(x);

// Iteration count
let k = 0;

console.log("System simulation ready");


System simulation ready


In [13]:
// Execute system simulation *once*
x = step(x, tBar);
k += 1;

// Statistics
reportStats(k, x, t);


Tag index k=1


t(k)= 0
x(k) = earliest possible firing times = [0, 0.1, 0.2]
lag(k) = [0, 0.1, 0.2]


In [18]:
// Execute system simulation *repeatedly*
// Bump the next logical time (scalar), then blast into vector
t = t + sensorPeriod;
tBar = [[t, t, t]];

// Step the system from k to k+1
x = step(x, tBar);
k += 1;

// Statistics
reportStats(k, x, t);


Tag index k=6
t(k)= 5
x(k) = earliest possible firing times = [5, 5.1, 5.2]
lag(k) = [0, 0.09999999999999964, 0.20000000000000018]
